In [ ]:
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth - Works only with Nvidia Graphics - 2022
# Prevent from geeting all GPU RAM's by tensorflow
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

1.2 Remove all Iamges with wrong extensions etc.

In [ ]:
import cv2
import imghdr
import shutil

In [ ]:
data_dir = 'data_v2'

In [ ]:
image_extensions = ['jpeg', 'jpg', 'bmp', 'png'] # List of available extensions

In [ ]:
# Check for trash data - wrong extensions
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_extensions:
                print(f'Image not in extensions list {image_path}')
                os.remove(image_path)
        except Exception as e:
            print(f'Issue with image {image_path}')
            

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Loading Data - Create datasets with labels (subdirectory names). Split for n-counts of batches with size of 32 (default)
data = tf.keras.utils.image_dataset_from_directory('data_v2', image_size=(224,224)) 

# It is working as Generator - do not loading files into memory
# Automaticly resize images - unified size of image 256x256 and create batches of images to maintain better prformance. 
# If our PC got not as much memory on GPU as it is needed, we can change size of batch ora change size of images 

In [ ]:
tf.keras.utils.image_dataset_from_directory?

In [ ]:
data_iterator = data.as_numpy_iterator() 

In [ ]:
batch = data_iterator.next() # This line grab the batches from pipeline - again and again and again

In [ ]:
# Images represented as numpy arrays
len(batch) # Print 2 - one for images in batches in te shape of numpy array
batch[0] # Print values from first position in batch
batch[0].shape # Show how many images are in a batch - in this case 32 it can be modified

In [ ]:
batch[1] # Labels of the images taken from directors contained in main directory

In [ ]:
fig, ax = plt.subplots( ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

    # Quick check which label is for which picture. In this case 0 - happy, 1 - sad


2. Preprocess Data

2.1 Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y)) # Allows create transformation in pipline

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[0].min()

In [ ]:
fig, ax = plt.subplots( ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

2.2 Split Data to Training and Testing 

In [ ]:
len(data) # How many batches have we got. This example: 7 batches with 32 images each

In [ ]:
# Split batches between training, validation and testing
# The sum of each has to equal the number of batches.
train_size = int(len(data)*.7)-2
val_size = int(len(data)*.2)+2
test_size = int(len(data)*.1)+1

# train and val data is used in trianing process. Training data allow us to trian deep learning network, validation is for model checking
# test batch is to check the results of training our network -> used at the end

In [ ]:
train_size

In [ ]:
val_size

In [ ]:
test_size

In [ ]:
batches_size = train_size + val_size + test_size

In [ ]:
batches_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# We allocate the batches of date to further training, validation and testing process. 
# .take() -> takes applied value of batches.
# .skip() -> skip applied value of batche.

# Important ! -> batches has to be shuffled before this process, and can not be shuffled after.
# It is, because we neeed to keep the order of taken batches to apply skip() function to work
# If we wanna shuffle data we has to back to: data = tf.keras.utils.image_dataset_from_directory('data'), and create new, shuffled batches

3. Deep model Creation

3.1 Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import keras
from keras.utils.vis_utils import plot_model
import graphviz
import pydot


In [ ]:
model = Sequential()

In [ ]:
# 3 - Convlutinal Layers, 1 - Flatten layer, 2 - Dense layers

# Conolutional Layer and MaxPooling Layer
# We got: 16 filters catching key features from picture. 3x3 -> 3px by 3px filters size, 1 stride -> moves the 3x3 squer one pixel each time
# activation = 'relu' -> the function we use to change input date. Relu changes all negative numbers to 0, 
# and the positive numbers multipl by a special value. There are many of activation functions. 
# Non-linear deep learning models are overall better, because deep learning is not linear method
# input_shape - size of images go into model. The sizes has been changed earlier in the program

model.add(Conv2D(16, (3,3), 1, activation="relu", input_shape=(224,224,3)))
model.add(MaxPooling2D()) 
# MaxPoooling2D returns max value, found after activation function. It will condense and scale down the input image. It takes the vlaues from
# region. In default (2,2) -> reduce size by half


model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

# Number of filters form the channel Value,but we need one chanel value.
# Flatten layer Create one-dimensional dataset, which is required input in Dense layer
model.add(Flatten())


# So we got one-dimensional 256 outputs after Dense function, and later we get 1 output.
# Dense layers are the fully connected layers in the Keras framework,
# We got 256 neurons, and we feed them into Dense layer with relu activation.
# After the second Dense operation we expect the value between 0 and 1 -> due to sigmoid chracteristics
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# W have t ocompile our model. We use 'adam' optimizer (one of many)
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary() # Allow us to see how the model is converting the input data

In [ ]:
# Print network flow chart
# tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

3.2 Train

In [ ]:
logdir = "logs"

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])
# model.fit(): takes training date (dividie earlier in the code), epoches -> how many times we wnat to train our model 
# -> one epoch is runinng over entire training set of data
# the nwe pass our validation vlaues, so we can know how great performance has our model in real-time domain

3.3 Plot Performance

In [ ]:
hist.history

In [ ]:
# Cause we saved information in hist variable, we cna now plot some interesting plots
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='Train loss')
plt.plot(hist.history['val_loss'], color='orange', label='Validation loss')
fig.suptitle('Loss', fontsize=15)
plt.xlabel('Iteration', fontsize=10)
plt.ylabel('Loss value', fontsize=10)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='Train accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='Validation accuracy')
fig.suptitle('Accuracy', fontsize=15)
plt.xlabel('Iteration', fontsize=10)
plt.ylabel('Accuracy value', fontsize=10)
plt.legend(loc='upper left')
plt.show()

4. Evaluate Performance

4.1 Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    ythat = model.predict(X)
    precision.update_state(y, ythat)
    recall.update_state(y, ythat)
    accuracy.update_state(y, ythat)
    

In [ ]:
print(f'Precision: {precision.result().numpy()}, Recall: {recall.result().numpy()}, Accuracy: {accuracy.result().numpy()}')

4.2 Test

In [ ]:
#img = cv2.imread('pain_test.JPG')
img = cv2.imread('test_Data/neutral_test_v2.JPG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig_3 = plt.figure()
plt.imshow(img)
fig_3.suptitle('test title', fontsize=20)
plt.xlabel('xlabel', fontsize=18)
plt.ylabel('ylabel', fontsize=16)

plt.show()

In [ ]:
# We resized all batches of data to 224x224 size. The input data has to be the same size !!!
resize_img = tf.image.resize(img, (224,224))
plt.imshow(resize_img.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize_img/255, 0)) 
# The model expect the batch of data not a single image, so we have to wrap data into batch (deeper into the list)

In [ ]:
yhat
#In this case 0 - happy, 1 - sad

In [ ]:
if yhat < 0.5:
    print('We are Normal Normal People, Yuhu :) !!!')
else:
    print("Hello Pain My Old Friend :( ")

5. Save model

In [ ]:
from tensorflow.keras.models import load_model
import os
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
# Save model to file
# model.save(os.path.join('models', 'pain_detection_model_v2.h5'))

6. Reuse the model

In [ ]:
# Load model from dir
new_model = load_model(os.path.join('models', 'pain_detection_model_baseCNN.h5'))

In [ ]:
input_img = cv2.imread('test_Data/neutral_test_v1.JPG')
input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
plt.imshow(input_img)
plt.show()

In [ ]:
resized_img = tf.image.resize(input_img, (256, 256))
plt.imshow(resized_img.numpy().astype(int))
plt.show()

In [ ]:
yhat_new = new_model.predict(np.expand_dims(resized_img/255, 0))

In [ ]:
yhat_new

In [ ]:
if yhat_new < 0.5:
    print('We are Normal Normal People, Yuhu :) !!!')
    plt.imshow(resized_img.numpy().astype(int))
else:
    print("Hello Pain My Old Friend :( ")
    plt.imshow(resized_img.numpy().astype(int))